# R notebook to create 10X formatted data for performance benchmarking

In [1]:
suppressPackageStartupMessages({

library(magrittr)
library(data.table)
library(dplyr)
library(scRepertoire)
library(hash)
library(purrr)

})

## Load Example Data

We'll be using a rowbinded and boostrapped version of the scRepertoire example raw TCR data to iteratively create larger and larger datasets

In [10]:
raw_10x_data <- get(data("contig_list", package = "scRepertoire")) %>%
    purrr::imap(~ {
        .x$barcode <- paste0(.y, "_", .x$barcode)
        .x
    }) %>%
    dplyr::bind_rows() %>%
    as.data.table()
print(dim(raw_10x_data))
head(raw_10x_data)

[1] 55154    18


barcode,is_cell,contig_id,high_confidence,length,chain,v_gene,d_gene,j_gene,c_gene,full_length,productive,cdr3,cdr3_nt,reads,umis,raw_clonotype_id,raw_consensus_id
<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>
P17B_AAACCTGAGTACGACG-1,True,AAACCTGAGTACGACG-1_contig_1,True,500,TRA,TRAV25,None,TRAJ20,TRAC,True,True,CGCSNDYKLSF,TGTGGGTGTTCTAACGACTACAAGCTCAGCTTT,8344,4,clonotype123,clonotype123_consensus_2
P17B_AAACCTGAGTACGACG-1,True,AAACCTGAGTACGACG-1_contig_2,True,478,TRB,TRBV5-1,None,TRBJ2-7,TRBC2,True,True,CASSLTDRTYEQYF,TGCGCCAGCAGCTTGACCGACAGGACCTACGAGCAGTACTTC,65390,38,clonotype123,clonotype123_consensus_1
P17B_AAACCTGCAACACGCC-1,True,AAACCTGCAACACGCC-1_contig_1,True,506,TRA,TRAV38-2/DV8,None,TRAJ52,TRAC,True,True,CAYRSAQAGGTSYGKLTF,TGTGCTTATAGGAGCGCGCAGGCTGGTGGTACTAGCTATGGAAAGCTGACATTT,18372,8,clonotype124,clonotype124_consensus_1
P17B_AAACCTGCAACACGCC-1,True,AAACCTGCAACACGCC-1_contig_2,True,470,TRB,TRBV10-3,None,TRBJ2-2,TRBC2,True,True,CAISEQGKGELFF,TGTGCCATCAGTGAACAGGGGAAAGGGGAGCTGTTTTTT,34054,9,clonotype124,clonotype124_consensus_2
P17B_AAACCTGCAGGCGATA-1,True,AAACCTGCAGGCGATA-1_contig_1,True,558,TRA,TRAV12-1,None,TRAJ9,TRAC,True,True,CVVSDNTGGFKTIF,TGTGTGGTCTCCGATAATACTGGAGGCTTCAAAACTATCTTT,5018,2,clonotype1,clonotype1_consensus_2
P17B_AAACCTGCAGGCGATA-1,True,AAACCTGCAGGCGATA-1_contig_2,True,505,TRB,TRBV9,None,TRBJ2-2,TRBC2,True,True,CASSVRRERANTGELFF,TGTGCCAGCAGCGTAAGGAGGGAAAGGGCGAACACCGGGGAGCTGTTTTTT,25110,11,clonotype1,clonotype1_consensus_1


## Save uniformly sampled subsets of the dataset in 10X format

In [15]:
bootstrap_tcr_data <- function(raw_10x_data, n_barcodes, seed = n_barcodes, barcode_col = "barcode") {
  set.seed(seed)
  unique_bcs <- unique(raw_10x_data[[barcode_col]])
  n_unique <- length(unique_bcs)
  
  # If n_barcodes <= number of unique barcodes, simple sampling without replacement
  if (n_barcodes <= n_unique) {
    sampled_bcs <- sample(unique_bcs, size = n_barcodes, replace = FALSE)
    sampled_dt <- data.table(temp_join_col = sampled_bcs)
    join_on_expr <- setNames("temp_join_col", barcode_col)
    return(raw_10x_data[sampled_dt, on = join_on_expr])
  }
  
  # For larger samples, we'll create a more efficient approach
  # First, get indices of rows matching each unique barcode
  all_unique_data <- raw_10x_data[raw_10x_data[[barcode_col]] %in% unique_bcs]
  
  # Calculate how many complete sets and remainder
  n_complete_sets <- n_barcodes %/% n_unique
  remainder <- n_barcodes %% n_unique
  
  # Create result directly
  if (remainder == 0) {
    # Just full repetitions
    result <- all_unique_data[rep(seq_len(.N), times = n_complete_sets)]
  } else {
    # Full repetitions plus remainder
    remainder_indices <- sample(seq_len(nrow(all_unique_data)), remainder)
    result <- all_unique_data[c(rep(seq_len(.N), times = n_complete_sets), remainder_indices)]
  }
  
  # Add set indices for making barcodes unique
  result[, set_index := c(rep(1:n_complete_sets, each = nrow(all_unique_data)), 
                          rep(n_complete_sets + 1, remainder))]
  
  # Update the barcode column to ensure uniqueness
  result[, (barcode_col) := paste0(get(barcode_col), "_", set_index)]
  result[, set_index := NULL]  # Remove the temporary column
  
  return(result)
}

In [ ]:
sizes_to_benchmark <- c(
    1e3, 3e3, 6e3, 1e4, 3e4, 6e4, 1e5, 3e5, 6e5, 1e6
)

for (size in as.integer(sizes_to_benchmark)) {

    target_dir <- paste0("datasets/", size, "/")
    dir.create(target_dir, recursive = TRUE, showWarnings = FALSE)

    benchmark_data <- bootstrap_tcr_data(raw_10x_data, size)
    data.table::fwrite(
        benchmark_data,
        file = paste0(target_dir, "filtered_contig_annotation.csv")
    )
    message("saved dataset with cellcount = ", size, ", nrow = ", nrow(benchmark_data))
}

saved dataset with cellcount = 1000, nrow = 2088

saved dataset with cellcount = 3000, nrow = 6080

saved dataset with cellcount = 6000, nrow = 12194

saved dataset with cellcount = 10000, nrow = 20237

saved dataset with cellcount = 30000, nrow = 57968

saved dataset with cellcount = 60000, nrow = 115936

saved dataset with cellcount = 100000, nrow = 183904

saved dataset with cellcount = 300000, nrow = 607648

saved dataset with cellcount = 600000, nrow = 1215296

saved dataset with cellcount = 1000000, nrow = 2006848

